# Application d'analyse des données de Formule 1 saison 2022
celle que je peux récupérer en tout cas

Cette application est réutilisable pour les prochaines saisons

## Imports

In [56]:
import pandas as pd
import numpy as np
import cufflinks as cf
import plotly.express as px
import chart_studio.plotly as py
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import json

## Backend
dans cette partie, je fais tous ce qui ne relève pas de l'affichage des graphs.
Cela permet d'avoir directement toutes les données d'un coup pour ensuite juste avoir a
choisir le graph a afficher

In [74]:
# importation des données
data_all = pd.read_csv("all.csv", sep=",")
data_all["index"] = data_all.groupby("pilot").cumcount()
data_all["index"] += 1

# dataset des points cumulés par pilote
data_points = data_all[["pilot", "points"]].copy()
data_points["cum_points"] = data_points.groupby("pilot").cumsum()
data_points["index"] = data_points.groupby("pilot").cumcount()

# extraction des pilotes par écurie
with open("teams.json") as jsonfile:
    teams_pilot = json.load(jsonfile)
data_team_points = {}
for team in teams_pilot.keys():
    data_team_points[team] = []
    
# pour chaque équipe on fait la somme des points de chaque pilote de l'équipe
for team in teams_pilot.keys():
    s = 0
    j = 0
    for i in range(data_points.shape[0]):
        if data_points.iloc[i]["pilot"] in teams_pilot[team]:
            s += data_points.iloc[i]["points"]
            j += 1
        if j == len(teams_pilot[team]):
            j = 0
            data_team_points[team].append(s)
data_team_points = pd.DataFrame(data_team_points)
test = []
for i in range(data_team_points.shape[0]):
    for team in teams_pilot.keys():
        test.append([team, data_team_points.iloc[i][team]])
data_team_points = pd.DataFrame(test, columns=["team", "points"])
data_team_points["index"] = data_team_points.groupby("team").cumcount()

## Frontend
Ici les affichages seront séparé pour pouvoir choisir ce qu'on affiche

### Test data import

In [67]:
print(data_all.head())

  pilot  avg_speed  best_lap  grid  points  result      time  Unnamed: 7  \
0   ham    202.469    96.228   5.0      15     3.0  5863.259         NaN   
1   rus    202.313    96.302   9.0      12     4.0  5864.795         NaN   
2   ver    204.140    95.440   2.0       0    19.0       NaN         NaN   
3   per    202.762    96.089   4.0       0    18.0       NaN         NaN   
4   lec    206.018    94.570   1.0      26     1.0  5853.584         NaN   

   index  
0      0  
1      0  
2      0  
3      0  
4      0  


### nuage de point
un graph avec la position sur la grille en x, et le classement en y. cela permet de predire si l'ordre de la grille est conservé

In [63]:
px.scatter(data_all, x="grid", y="result", color="pilot", title="Classement par pilote", template="plotly_dark")

toujours un nuage de point, mais avec les temps totaux en x et temps du meilleur tour en y

In [75]:
px.scatter(data_all, x="time", y="best_lap", color="pilot", title="Meilleur temps par pilote", template="plotly_dark", size="index")

### Heatmap

Ici je vais faire toutes les heatmap.

la première ci-dessous est grid/result

In [54]:
px.density_heatmap(data_all, x="grid", y="result", title="resultat en fonction de la grille", template="plotly_dark")

### Evolutions

Ici c'est les courbes d'evolutions au cours de la saison

In [55]:
px.line(data_points, x="index", y="cum_points", color="pilot", title="Cumul des points par pilote", template="plotly_dark", markers=True)

Et même chose mais pour les écuries

In [47]:
px.line(data_team_points, x="index", y="points", color="team", title="Cumul des points par équipe", template="plotly_dark", markers=True)